In [1]:
import pandas as pd

df = pd.read_csv("Data.csv")
print(df)

       Total Household Income              Region  Staple Food Expenditure  \
0                        9370                 CAR                     1736   
1                        3540                 CAR                      718   
2                        1478                 CAR                     1281   
3                        2199                 CAR                     1273   
4                        4732                 CAR                     1386   
...                       ...                 ...                      ...   
41539                    3820  XII - SOCCSKSARGEN                      906   
41540                    3850  XII - SOCCSKSARGEN                      159   
41541                    4138  XII - SOCCSKSARGEN                     1076   
41542                    4231  XII - SOCCSKSARGEN                     1206   
41543                    4181  XII - SOCCSKSARGEN                     1676   

                 Source of Income  Agricultural Household indic

In [2]:
df.columns

Index(['Total Household Income', 'Region', 'Staple Food Expenditure',
       'Source of Income', 'Agricultural Household indicator',
       'Meat Expenditure', 'Seafood Expenditure', 'Leisure Expenditure',
       'Alcohol Expenditure ', 'Tobacco Expenditure', 'Medical Expenditure',
       'Transportation Expenditure', 'Communication Expenditure',
       'Utilities Expenditure', 'Housing Tax', 'Education Expenditure',
       'Crop Farming Expenditure', 'Household Head Gender',
       'Household Head Age', 'Household Head Marital Status',
       'Household Head Highest Grade Completed', 'Education Rank',
       'Household Head Job Indicator', 'Household Head Occupation',
       'Household Head Class of Worker', 'Occupation group',
       'Type of Household', 'Total Number of Family members',
       'Members with age less than 5 year old',
       'Members with age 5 - 17 years old',
       'Total number of family members employed', 'House Floor Area',
       'House Age', 'Number of bedroo

In [3]:
selected_columns = ["Total Household Income","Region","Total Food Expenditure","Bread and Cereals Expenditure","Total Rice Expenditure","Meat Expenditure","Total Fish and marine products Expenditure","Fruit Expenditure","Vegetables Expenditure","Restaurant and hotels Expenditure", "Alcohol Beverages Expenditure","Tobacco Expenditure","Clothing, Footwear and Other Wear Expenditure","Housing and water Expenditure","Imputed House Rental Value","Medical Care Expenditure", "Transportation Expenditure", "Communication Expenditure", "Education Expenditure", "Miscellaneous Goods and Service Expenditure","Special Occasions Expenditure","Crop Farming and Gardening expenses","Electricity","Number of Television","Number of CD/VCD/DVD","Number of Refrigerator/Freezer","Number of Washing Machine","Number of Airconditioner","Number of Car, Jeep, Van","Number of Landline/wireless telephones","Number of Cellular phone","Number of Personal Computer","Number of Stove with Oven/Gas Range","Number of Motorcycle/Tricycle"]
df_selected = df["New Dataset"]

KeyError: 'New Dataset'

In [4]:
df_filtered = pd.DataFrame(df, columns=selected_columns)
print(df_filtered)

       Total Household Income              Region  Total Food Expenditure  \
0                        9370                 CAR                     NaN   
1                        3540                 CAR                     NaN   
2                        1478                 CAR                     NaN   
3                        2199                 CAR                     NaN   
4                        4732                 CAR                     NaN   
...                       ...                 ...                     ...   
41539                    3820  XII - SOCCSKSARGEN                     NaN   
41540                    3850  XII - SOCCSKSARGEN                     NaN   
41541                    4138  XII - SOCCSKSARGEN                     NaN   
41542                    4231  XII - SOCCSKSARGEN                     NaN   
41543                    4181  XII - SOCCSKSARGEN                     NaN   

       Bread and Cereals Expenditure  Total Rice Expenditure  \
0          

In [5]:
print(df.head())


   Total Household Income Region  Staple Food Expenditure Source of Income  \
0                    9370    CAR                     1736             Wage   
1                    3540    CAR                      718             Wage   
2                    1478    CAR                     1281             Wage   
3                    2199    CAR                     1273             Wage   
4                    4732    CAR                     1386             Wage   

   Agricultural Household indicator  Meat Expenditure  Seafood Expenditure  \
0                                 0               441                  300   
1                                 0               311                  198   
2                                 1               139                   46   
3                                 0               195                  193   
4                                 0               328                  202   

   Leisure Expenditure  Alcohol Expenditure   Tobacco Expendit

In [7]:
df_filtered.to_csv('filtered_data.csv', index=False)